In [6]:
from quant_utils.db_conn import DB_CONN_JJTG_DATA

import pandas as pd

file_path = "C:/Users/Wilcoxon/Desktop/配置excel/组合配置信息/"
file_list = [
    'portfolio_benchmark.xlsx',
    'portfolio_info.xlsx',
    'portfolio_constraint.xlsx',
    'portfolio_products_weights.xlsx',
]

for file in file_list:
    df = pd.read_excel(file_path + file)
    if "TICKER_SYMBOL" in df.columns:
        df["TICKER_SYMBOL"] = (
            df["TICKER_SYMBOL"]
            .astype(int)
            .astype(str)
            .str.rjust(6, "0")
        )
        df["TICKER_SYMBOL"] = df["TICKER_SYMBOL"].str.rjust(6, "0")
        df.loc[df["TICKER_SYMBOL"] == "000000", "TICKER_SYMBOL"] = None

    if "ALTERNATIVE_TICKER_SYMBOL" in df.columns:
        df["ALTERNATIVE_TICKER_SYMBOL"].fillna(0, inplace=True)
        df["ALTERNATIVE_TICKER_SYMBOL"] = (
            df["ALTERNATIVE_TICKER_SYMBOL"]
            .astype(int)
            .astype(str)
            .str.rjust(6, "0")
        )
        df.loc[df["ALTERNATIVE_TICKER_SYMBOL"] == "000000", "ALTERNATIVE_TICKER_SYMBOL"] = None
    DB_CONN_JJTG_DATA.upsert(df, table=file.split('.')[0])

C:\Users\Wilcoxon\AppData\Local\Temp\ipykernel_24328\2857444691.py:26: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["ALTERNATIVE_TICKER_SYMBOL"].fillna(0, inplace=True)


In [5]:
import pandas as pd
template_path = "E:/基金投顾自动化/"
trade_time = '20241021'
data = pd.read_excel(
    template_path + "基金投顾业务实时数据.xlsx",
    sheet_name="基金投顾客户级明细报表",
    engine="openpyxl",
)
# data = data[~data["组合名称"].isin(self.black_list)]
data["委托时间"] = pd.to_datetime(data["委托时间"]).dt.strftime("%Y%m%d")
data["委托金额"] = data["委托金额"] / 10000
data = data[data["委托时间"] == trade_time]
data = data[
    data["交易类型"].isin(
        [
            "减少投资",
            "解约"
        ]
    )
]
data = data[data["委托时间"] >= trade_time]
data = data[
    data["交易类型"].isin(
        [
            "减少投资",
            "解约"
        ]
    )
]
# data = data.query(f"委托金额 >= {self.min_amount}")
data = (
    data.groupby(by=["分公司", "委托时间", "客户编号", "组合名称"])["委托金额"]
    .sum()
    .reset_index()
    .sort_values(by="委托金额", ascending=False)
)
data.index = range(0, len(data))

d:\miniforge3\envs\my312\Lib\site-packages\openpyxl\styles\stylesheet.py:241: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [6]:
df = data.groupby(by=["组合名称"])["委托金额"].sum().reset_index()

In [7]:
df.sort_values(by="委托金额", ascending=False)

,组合名称,委托金额
5,知己优选-货币小福星,413.306784
4,知己优选-短债增强,164.765085
2,知己优选-均衡小确幸,13.152858
6,知己优选-进取全明星,6.120665
3,知己优选-月月享,6.035504
1,知己优选-500增强,0.024804
7,知己目标盈006期,0.010365
0,知己优选-300增强,0.008546


In [8]:
df["委托金额"].sum()

603.424611